국어 (독서, 문학)의 주제 선정

1. **시가복합 사용**:
   - 선정된 문항 9개: [29742, 29743, 29745, 29744, 29757, 29758, 29756, 29754, 29755]
   - 학생 수: 52명

2. **갈래 복합 사용**:
   - 선정된 문항 8개: [33028, 33031, 33029, 33030, 33047, 33050, 33046, 33049] 
   - 학생 수: 60명
   
   **혹은**
   
   - 선정된 문항 10개: [35662, 35660, 35663, 35661, 35683, 35680, 35681, 35682, 35679, 35678]
   - 학생 수: 35명

3. **기존의 현대서사 사용**:
   - 선정된 문항 8개: [33028, 33031, 33029, 33030, 33051, 33054, 33052, 33053] 
   - 학생 수: 59명


In [1]:
import pandas as pd
import numpy as np
import random
import torch
import argparse
from scipy.stats import mode
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import yaml
import json
import warnings
import itertools
import os
from collections import Counter, defaultdict
warnings.filterwarnings('ignore')

In [2]:
from types import SimpleNamespace

# Define your parameters as a dictionary
parameters_dict = {
    # "subjects": '생명과학Ⅰ',
    # "subjects": '화학Ⅰ',
    # "subjects": '독해,듣기',
    # "subjects": '화학Ⅱ',
    "subjects": '지구과학Ⅰ',
    # "subjects": '통합과학',
    # "subjects": '윤리와 사상',
    # "subjects": '수학Ⅰ,수학Ⅱ',
    # "subjects": '물리학Ⅱ',
    # "subjects": '통합사회',
    # "subjects": '정치와 법',
    # "subjects": '독서,문학',
    # "subjects": '한국사',
    # "subjects": '사회?문화',
    # "subjects": '동아시아사',
    # "subjects": '한국사',
    "num_problems": 10,
    'set_type':['EC040003','EC040012'],
    # ['EC040003','EC040014'], # 'EC040003': '사회', 'EC040013': '현대 서사' -> EC040011 시가 복합 or 갈래 복합 EC040012
    # {'EC040001': '독서이론', 'EC040002': '인문', 'EC040003': '사회', 'EC040004': '과학', 'EC040005': '기술', 'EC040006': '예술', 'EC040007': '융합', 'EC040008': '고전 시가', 'EC040009': '고전 서사', 'EC040010': '고전 교술', 'EC040011': '시가 복합', 'EC040012': '갈래 복합', 'EC040013': '현대 시', 'EC040014': '현대 서사', 'EC040015': '현대 교술', 'EC040016': '현대 극', 'EC040017': '화법', 'EC040018': '작문', 'EC040019': '화법+작문', 'EC040020': '언어','EC040021': '매체', 'EC040022': '언어+매체'}
    "seed": 42,
    "original_data_path": '../../../data/full_data',
    "same_standard":  'middle_category',  # 'small_category', #
    "IQR_range": 1.5
}

def set_seed(seed):
    random.seed(seed)  
    np.random.seed(seed)  
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  

# Convert dictionary to SimpleNamespace
parameters = SimpleNamespace(**parameters_dict)
set_seed(parameters.seed)

In [3]:
###### import original datasets
category = pd.read_csv(f'{parameters.original_data_path}/perfectexam_category_202404031323.csv', encoding='cp949')
problem = pd.read_csv(f'{parameters.original_data_path}/perfectexam_problem_202404031323.csv', encoding='cp949')
member_exam_answer = pd.read_csv(f'{parameters.original_data_path}/perfectexam_member_exam_answer_202404031323.csv', encoding='cp949')
member_exam = pd.read_csv(f'{parameters.original_data_path}/perfectexam_member_exam_202404031323.csv', encoding='cp949')
##### 여기서 시험 범위 제한 가능 ######
exam_in_range = pd.read_csv(f'{parameters.original_data_path}/perfectexam_exam_202404031323.csv', encoding='cp949')

category_simple = category[['code', 'code_name']]
problem_simple = problem[['seq', 'subject', 'big_category', 'middle_category','small_category','ref_seq','point','answer_rate',"set_type", "type1", "type2", "num1", "ref_num", "represent_yn"]]
member_exam_answer_simple = member_exam_answer[['member_idx', 'member_exam_seq', 'problem_seq', 'answer_dttm','correct_yn']]
member_exam_simple = member_exam[['seq', 'exam_seq', 'title']]
first_merge = member_exam_answer_simple.merge(member_exam_simple, left_on='member_exam_seq', right_on='seq', how='left').drop(columns='seq')
second_merge = first_merge[first_merge['exam_seq'].isin(exam_in_range['seq'])]
third_merge = second_merge.merge(exam_in_range[['seq', 'title', 'explain_text','open_dttm']], left_on='exam_seq', right_on='seq', how='left').drop(columns=['seq', 'exam_seq', 'title_x'])
member_exam_answer_in_range = third_merge.merge(problem_simple, left_on='problem_seq', right_on='seq', how='left').drop(columns='seq')
category_dict = pd.Series(category_simple['code_name'].values, index=category_simple['code']).to_dict()
for column in ['subject', 'big_category', 'middle_category','small_category']:
    member_exam_answer_in_range[column] = member_exam_answer_in_range[column].map(category_dict)

###### filter by subjects
subjects_list = parameters.subjects.split(',')
print('subjects_list',subjects_list)
filtered_data = member_exam_answer_in_range[member_exam_answer_in_range['subject'].isin(subjects_list)]


subjects_list ['지구과학Ⅰ']


First attemps

In [4]:
###### get first attemps
filtered_data_sorted = filtered_data.sort_values(by=['member_idx', 'problem_seq', 'answer_dttm'])

################### get last attempts
first_attempts = filtered_data_sorted.groupby(['member_idx', 'problem_seq']).last().reset_index()

###### get new_seq (같은 문제로 볼 기준 정하기 -> standardized_problem_seq 정해주기)
def generate_new_seq(row, standard):
    standards = {
        'big_category': ['subject', 'big_category', 'point'],
        'middle_category': ['subject', 'big_category', 'middle_category', 'point'],
        'small_category': ['subject', 'big_category', 'middle_category', 'small_category', 'point']}
    if standard in standards:
        return '-'.join(str(row[col]) for col in standards[standard])
    else:
        raise ValueError(f"Unsupported standard: {standard}")

first_attempts['new_seq_middle'] = first_attempts.apply(generate_new_seq, axis=1, standard=parameters.same_standard)
representative_problem_seq = (first_attempts.groupby('new_seq_middle')['problem_seq'].apply(lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0]).reset_index())
problem_seq_map = dict(zip(representative_problem_seq['new_seq_middle'], representative_problem_seq['problem_seq']))
first_attempts['standardized_problem_seq'] = first_attempts['new_seq_middle'].map(problem_seq_map)
first_attempts.iloc[1:3,8:]

,subject,big_category,middle_category,small_category,ref_seq,point,answer_rate,set_type,type1,type2,num1,ref_num,represent_yn,new_seq_middle,standardized_problem_seq
1,지구과학Ⅰ,지구의 역사,퇴적암과 퇴적 구조,"퇴적암: 퇴적암의 생성, 퇴적암의 분류",NaN,3.0,68.0,None,EC010031,None,4.0,NaN,N,지구과학Ⅰ-지구의 역사-퇴적암과 퇴적 구조-3.0,30118
2,지구과학Ⅰ,별과 외계 행성계,외계 생명체 탐사,생명 가능 지대,NaN,2.0,85.0,None,EC010031,None,5.0,NaN,N,지구과학Ⅰ-별과 외계 행성계-외계 생명체 탐사-2.0,30119


In [5]:
##### new_seq별 너무 accuracy가 다른 response 제거   
def remove_outliers(df):
    Q1 = df['answer_rate'].quantile(0.25)
    Q3 = df['answer_rate'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - parameters.IQR_range * IQR
    upper_bound = Q3 + parameters.IQR_range * IQR
    return df[(df['answer_rate'] >= lower_bound) & (df['answer_rate'] <= upper_bound)]

first_attempts_after = first_attempts.groupby('new_seq_middle').apply(remove_outliers).reset_index(drop=True)
columns_to_drop_candidate = ['answer_dttm', 'answer_time(문제풀이 시간)', 'ref_seq','member_exam_seq'] # 'title_y'
columns_to_drop = [col for col in columns_to_drop_candidate if col in first_attempts_after.columns]
preprocessing_done = first_attempts_after.drop(columns=columns_to_drop)
preprocessing_done['correct_yn'] = preprocessing_done['correct_yn'].map({'Y': 1, 'N': 0})
preprocessing_done = preprocessing_done.applymap(lambda x: x.replace('・', '?').replace('∙', '?') if isinstance(x, str) else x) ###############. 기호 대신 ? 추가 ###############
preprocessing_done[:5]

,member_idx,problem_seq,correct_yn,title_y,explain_text,open_dttm,subject,big_category,middle_category,small_category,point,answer_rate,set_type,type1,type2,num1,ref_num,represent_yn,new_seq_middle,standardized_problem_seq
0,447,29055,0,2022학년도 10월 고3 학력평가 과학탐구 (지구과학Ⅰ),2022년 10월 시행 | 20문제,2022-11-24 0:00,지구과학Ⅰ,대기와 해양의 변화,기압과 날씨 변화,전선과 날씨,2.0,71.0,None,EC010031,None,9.0,NaN,N,지구과학Ⅰ-대기와 해양의 변화-기압과 날씨 변화-2.0,22653
1,642,23804,0,2022학년도 4월 고3 학력평가 과학탐구 (지구과학Ⅰ),2022년 4월 시행 | 20문제,2022-10-19 0:00,지구과학Ⅰ,대기와 해양의 변화,기압과 날씨 변화,전선과 날씨,2.0,54.0,None,EC010031,None,10.0,NaN,N,지구과학Ⅰ-대기와 해양의 변화-기압과 날씨 변화-2.0,22653
2,642,29055,1,2022학년도 10월 고3 학력평가 과학탐구 (지구과학Ⅰ),2022년 10월 시행 | 20문제,2022-11-24 0:00,지구과학Ⅰ,대기와 해양의 변화,기압과 날씨 변화,전선과 날씨,2.0,71.0,None,EC010031,None,9.0,NaN,N,지구과학Ⅰ-대기와 해양의 변화-기압과 날씨 변화-2.0,22653
3,1230,22655,1,2021학년도 3월 고3 학력평가 과학탐구 (지구과학Ⅰ),2021년 3월 시행 | 20문제,2022-10-19 0:00,지구과학Ⅰ,대기와 해양의 변화,기압과 날씨 변화,온대 저기압의 날씨,2.0,58.0,None,EC010031,None,5.0,NaN,N,지구과학Ⅰ-대기와 해양의 변화-기압과 날씨 변화-2.0,22653
4,1765,23268,1,2020학년도 6월 고2 학력평가 과학탐구 (지구과학Ⅰ),2020년 6월 시행 | 20문제,2022-10-19 0:00,지구과학Ⅰ,대기와 해양의 변화,기압과 날씨 변화,온대 저기압의 날씨,2.0,38.0,None,EC010031,None,14.0,NaN,N,지구과학Ⅰ-대기와 해양의 변화-기압과 날씨 변화-2.0,22653


In [6]:
cumul_data = {}
for subject in subjects_list:
    file_path = f'{parameters.original_data_path}/cumul_{subject}.csv'
    try: 
        cumul_data[subject] = pd.read_csv(file_path, encoding='cp949')
    except:
        cumul_data[subject] = pd.read_csv(file_path, encoding='UTF-8')

# Load JSON category structures
category_structures = {}
for subject in subjects_list:
    with open(f'{parameters.original_data_path}/{subject}_middle_freq_list.json', 'r', encoding='UTF-8') as file:
        category_structures[subject] = json.load(file)

category_structures


{'지구과학Ⅰ': {'지권의 변동': {'마그마 생성': 1},
  '지구의 역사': {'퇴적암과 퇴적 구조': 1},
  '대기와 해양의 변화': {'기압과 날씨 변화': 1, '태풍': 1},
  '대기와 해양의 상호 작용': {'엘니뇨와 라니냐': 1, '기후 변화의 요인': 1},
  '별과 외계 행성계': {'별의 물리량': 1, '외계 행성계 탐사': 1, '외계 생명체 탐사': 1},
  '외부 은하와 우주 팽창': {'빅뱅 우주론': 1}}}

In [7]:
def select_frequent_problems_general(preprocessing_done, cumul_df, category_structure): # category_structure: { '간접 쓰기': {'글의 순서 파악': 1, '문장의 위치 파악': 1},}
    set_problems_dict = {} # use this to check if the problem per category is correctly chosen
    problems_nested_list = [] 
    for big_category, middle_categories in category_structure.items():
        for middle_category, freq in middle_categories.items(): # big_category 지권의 변동 middle_category 마그마 생성 freq 1
            filtered_data = cumul_df[(cumul_df['big_category'] == big_category) & 
                                    (cumul_df['middle_category'] == middle_category)]
            print('big_category',big_category,'middle_category',middle_category)
            print('\nfiltered_data',filtered_data.head(2))
            
            value_counts = filtered_data['small_category'].value_counts()
            categories = value_counts.index.tolist()
            selections = {category: 0 for category in categories}
            # most_frequent_small_categories = []
            for _ in range(freq):
                max_category = max(categories, key=lambda x: value_counts[x] / (selections[x] + 1))
                selections[max_category] += 1
                # most_frequent_small_categories.append(max_category)
                
                filtered_preprocessing_done = preprocessing_done[(preprocessing_done['big_category'] == big_category) & (preprocessing_done['middle_category'] == middle_category)]
                print('\nfiltered_preprocessing_done',filtered_preprocessing_done.head(2))
                problems_in_category = filtered_preprocessing_done[filtered_preprocessing_done['small_category'] == max_category]
                print('\nproblems_in_category',problems_in_category.head(2))
                top_problems = problems_in_category.groupby('standardized_problem_seq').size().nlargest(5).index.tolist()
                print('\ntop_problems',top_problems)
                # if max_category in set_problems_dict:
                #     set_problems_dict[max_category].extend(top_problems)
                # else:
                #     set_problems_dict[max_category] = top_problems  # set_problems_dict {'탄소 화합물의 유용성': [31413, 24875]}
                problems_nested_list.append(top_problems) # problems_nested_list [[31413, 24875], [31710, 24027], [24077, 30670], [32912, 25040], [25474, 25406], [30667, 33329], [33327, 24937], [25445, 24643], [25451, 32068], [24489, 24633]]
    return problems_nested_list

def check_set_problems(problem_num):
    filtered_titles = preprocessing_done[(preprocessing_done['problem_seq'] == problem_num) &
                                    (preprocessing_done['title_y'].str.contains('학년도'))]
    try:
        unique_title_y = filtered_titles['title_y'].mode()[0]
        unique_refnum = filtered_titles['ref_num'].unique()
        final_unique_problem_seqs = preprocessing_done[
            (preprocessing_done['title_y'] == unique_title_y) &
            (preprocessing_done['ref_num'].isin(unique_refnum))]['problem_seq'].unique()
        return final_unique_problem_seqs.tolist()
    except:
        return []
    
def select_frequent_problems_korean(preprocessing_done, category_structure):  
    problems_nested_list = [] 
    for temp_value, set_categories in category_structure.items():
        for set_value, freq in set_categories.items():
            problems_in_category = preprocessing_done[preprocessing_done['set_type'] == set_value]
            temp_set_list = []
            top_problems = problems_in_category.groupby('problem_seq').size().nlargest(freq*5).index.tolist()
            for problem in top_problems:
                problem_set = check_set_problems(problem)  #  [31413, 24875]
                # Check if any element in problem_set is already in problems_nested_list
                if not any(element in sublist for sublist in problems_nested_list for element in problem_set):
                    temp_set_list.append(problem_set)
                if len(temp_set_list) >= freq:
                    break
            problems_nested_list.append(temp_set_list)
    return problems_nested_list

def select_frequent_problems_english(preprocessing_done, cumul_df, category_structure):  
    problems_nested_list = []
    for big_category, freq_dict in category_structure.items():
        for middle_category, freq in freq_dict.items():
            problems_in_category = preprocessing_done[(preprocessing_done['big_category'] == big_category) & 
                                    (preprocessing_done['middle_category'] == middle_category)]
            top_problems = problems_in_category.groupby('standardized_problem_seq').size().nlargest(freq*5).index.tolist()
            problems_nested_list.append(top_problems) 
    return problems_nested_list

problems_list_tobe_combinationed = []
for subject, cumul_df in cumul_data.items(): 
    cumul_df = cumul_df.replace(to_replace=["・", "∙"], value="?", regex=True)
    category_structure = category_structures[subject]
    if subject in ['독서', '문학']: # 국어 라면
        # 국어는 small_category가 없음 & category가 아닌 set_type 사용 & 
        # 지문을 고려하기 위해 standard_problem_seq가 아닌 problem_seq 사용
        problems_nested_list=select_frequent_problems_korean(preprocessing_done, category_structure)
    
    elif subject in ['독해', '듣기']: # 영어라면
        # 영어도 small_category가 없음
        problems_nested_list=select_frequent_problems_english(preprocessing_done, cumul_df, category_structure)
    else:
        problems_nested_list=select_frequent_problems_general(preprocessing_done, cumul_df, category_structure)
    problems_list_tobe_combinationed.extend(problems_nested_list)
print('problems_list_tobe_combinationed',len(problems_list_tobe_combinationed), problems_list_tobe_combinationed) # nested list


big_category 지권의 변동 middle_category 마그마 생성

filtered_data    subject big_category middle_category small_category  freq
31   지구과학Ⅰ       지권의 변동          마그마 생성      마그마 생성 조건     9
35   지구과학Ⅰ       지권의 변동          마그마 생성   판 경계와 마그마 생성     8

filtered_preprocessing_done        member_idx  problem_seq  correct_yn                          title_y  \
19232         642        23800           1  2022학년도 4월 고3 학력평가 과학탐구 (지구과학Ⅰ)   
19233         642        30120           1   2023학년도 11월 고3 수능 과학탐구 (지구과학Ⅰ)   

              explain_text        open_dttm subject big_category  \
19232   2022년 4월 시행 | 20문제  2022-10-19 0:00   지구과학Ⅰ       지권의 변동   
19233  2022년 11월 시행 | 20문제  2022-11-28 0:00   지구과학Ⅰ       지권의 변동   

      middle_category small_category  point  answer_rate set_type     type1  \
19232          마그마 생성   판 경계와 마그마 생성    2.0         82.0     None  EC010031   
19233          마그마 생성   판 경계와 마그마 생성    2.0         68.0     None  EC010031   

      type2  num1  ref_num represent_yn          

## 밑에 코드 고치면 끝! 그리고 나중에 

In [8]:
def solved_all_problems(group, required_problems):
    solved_problems = set(group['standardized_problem_seq'])
    return solved_problems == set(required_problems)

def solved_all_problems_korean(group, required_problems):
    solved_problems = set(group['problem_seq'])
    return solved_problems == set(required_problems)

def process_combination(combination):
    if set(subjects_list) & {'독서', '문학'}: 
        ###
        # [([35662, 35660, 35663, 35661], [33051, 33054, 33052, 33053]),
        # ([29742, 29743, 29745, 29744], [31019, 31021, 31022, 31020])]

        # to
        # [[35662, 35660, 35663, 35661, 33051, 33054, 33052, 33053],
        #  [29742, 29743, 29745, 29744, 31019, 31021, 31022, 31020]]
        ###
        problem_combination = combination[0] + combination[1]
        if len(problem_combination) != len(set(problem_combination)): 
            return 0, problem_combination        
        filtered_df = preprocessing_done[preprocessing_done['problem_seq'].isin(problem_combination)]
        solved_all = filtered_df.groupby('member_idx').filter(lambda x: solved_all_problems_korean(x, problem_combination))
        return len(solved_all['member_idx'].unique()), problem_combination
    else:
        problem_combination = list(combination)
        if len(problem_combination) != len(set(problem_combination)):        
            return 0, problem_combination         
        filtered_df = preprocessing_done[preprocessing_done['standardized_problem_seq'].isin(problem_combination)]
        solved_all = filtered_df.groupby('member_idx').filter(lambda x: solved_all_problems(x, problem_combination))
        return len(solved_all['member_idx'].unique()), problem_combination

max_member_count = 0
all_top_standardized = []
combinations = list(itertools.product(*problems_list_tobe_combinationed))
with ProcessPoolExecutor(max_workers=32) as executor:
    results = executor.map(process_combination, combinations)
    for unique_member_count, problem_combination in results:
        # if unique_member_count > max_member_count:
        if (unique_member_count > max_member_count) & (len(problem_combination)==10):

            max_member_count = unique_member_count
            all_top_standardized = problem_combination
        ########
print('선정된 standard_problem_seq', len(all_top_standardized), all_top_standardized, 'max_member_counts', max_member_count)


if '독해' in subjects_list:
    for i, problem in enumerate(all_top_standardized):
        temp2_df = preprocessing_done[preprocessing_done['standardized_problem_seq'] == problem]
        if '세트 문항' in temp2_df['big_category'].values:
            set_problem_result = check_set_problems(problem)
            all_top_standardized = all_top_standardized[:i] + set_problem_result + all_top_standardized[i+1:]
            break


선정된 standard_problem_seq 10 [30120, 30118, 30122, 30121, 30131, 33371, 33381, 33383, 30119, 30125] max_member_counts 63


In [9]:
results

<generator object _chain_from_iterable_of_lists at 0x742d5ec6cac0>

In [10]:
##### 뽑힌 문제 확인 csv 저장 #####
results = all_top_standardized
selected_df = pd.DataFrame(results, columns=['Problem Seq'])
if set(subjects_list) & {'독서', '문학'}:
    calculate_ratio_df = selected_df.merge(preprocessing_done, left_on='Problem Seq', right_on='problem_seq', how='left')
else:
    calculate_ratio_df = selected_df.merge(preprocessing_done, left_on='Problem Seq', right_on='standardized_problem_seq', how='left')

remaining_details = preprocessing_done[['problem_seq','subject','big_category','middle_category','small_category','point','title_y','explain_text', 'open_dttm']]
# detailed_df = detailed_df[['Problem Seq', 'subject', 'big_category', 'middle_category', 'small_category', 'point', 'new_seq_middle']]
calculate_ratio_df = calculate_ratio_df[['Problem Seq', 'subject', 'big_category', 'middle_category', 'small_category', 'point', 'new_seq_middle']]
calculate_ratio_df = calculate_ratio_df.drop_duplicates()

# detailed_df.sort_values(['subject', 'big_category', 'middle_category', 'small_category'], inplace=True)
response_counts = preprocessing_done.groupby('new_seq_middle').size().rename('response_count')

correct_yn_counts = preprocessing_done.groupby('new_seq_middle')['correct_yn'].value_counts().unstack(fill_value=0)

correct_yn_counts = correct_yn_counts.rename(columns={0: 'incorrect_count', 1: 'correct_count'})
correct_yn_counts['response_summary'] = correct_yn_counts.apply(lambda x: f"0:{x['incorrect_count']}, 1:{x['correct_count']}", axis=1)
calculate_ratio_df = calculate_ratio_df.merge(response_counts, left_on='new_seq_middle', right_index=True, how='left')
calculate_ratio_df = calculate_ratio_df.merge(correct_yn_counts[['response_summary']], left_on='new_seq_middle', right_index=True, how='left')
# detailed_df = detailed_df[['Problem Seq', 'subject', 'title_y', 'explain_text',	'open_dttm', 'big_category', 'middle_category', 'small_category', 'point', 'response_count', 'response_summary']]

def replace_titles(group):
    try:
        filtered_group = group[group['explain_text'].notnull()]
        filtered_group['open_dttm'] = pd.to_datetime(filtered_group['open_dttm'])
        earliest_row = filtered_group.loc[filtered_group.groupby('title_y')['open_dttm'].idxmin()]
        earliest_title = earliest_row.loc[earliest_row['open_dttm'].idxmin(), 'title_y']
        group['title_y'] = earliest_title
    except IndexError:
        group['title_y'] = "Default Title"
    return group


# remaining_details join calculate_ratio_df on left problem_seq, right 'Problem Seq' , get  response_count	response_summary from calculate_ratio_df else from remaining_details
detailed_df = remaining_details.merge(
    calculate_ratio_df[['Problem Seq', 'response_count', 'response_summary']],
    how='left',  # Perform left join
    left_on='problem_seq',  # Left key
    right_on='Problem Seq'  # Right key
)

# Step 2: Drop the 'Problem Seq' column from the merged dataframe
detailed_df = detailed_df.drop(columns=['Problem Seq'])

# Step 3: Drop rows where 'response_count' is NaN
detailed_df = detailed_df.dropna(subset=['response_count'])

detailed_df.drop_duplicates(inplace=True)
detailed_df.head(1)

,problem_seq,subject,big_category,middle_category,small_category,point,title_y,explain_text,open_dttm,response_count,response_summary
457,30122,지구과학Ⅰ,대기와 해양의 변화,기압과 날씨 변화,온대 저기압의 날씨,3.0,2023학년도 11월 고3 수능 과학탐구 (지구과학Ⅰ),2022년 11월 시행 | 20문제,2022-11-28 0:00,975.0,"0:304, 1:671"


In [11]:
detailed_df_2= detailed_df.groupby('problem_seq').apply(replace_titles)
# detailed_df_2 = detailed_df_2.drop_duplicates(subset='problem_seq', keep='first')
detailed_df_2['open_dttm'] = pd.to_datetime(detailed_df_2['open_dttm'])
# detailed_df_2
idx = detailed_df_2.groupby(level='problem_seq')['open_dttm'].idxmin()
detailed_df_2 = detailed_df_2.loc[idx]
detailed_df_2 = detailed_df_2.reset_index(drop=True)
def calculate_response_ratio(summary):
    pairs = summary.split(', ')
    nums = [int(pair.split(':')[1]) for pair in pairs]
    total = sum(nums)
    ratios = [round(num / total, 2) for num in nums]
    return ratios
detailed_df_2['response_ratio'] = detailed_df_2['response_summary'].apply(calculate_response_ratio)

target_counts = Counter(all_top_standardized)
rows_to_add = []
for problem, required_count in target_counts.items():
    mask = detailed_df_2['problem_seq'] == problem
    current_count = mask.sum()  
    additional_needed = required_count - current_count 
    if additional_needed > 0:
        rows_to_duplicate = detailed_df_2[mask].copy() 
        for _ in range(additional_needed):
            top_sequences = preprocessing_done[preprocessing_done['standardized_problem_seq'] == problem]['problem_seq'].value_counts().nlargest(5)
            selected_seq = None
            for seq, _ in top_sequences.items():
                if seq not in detailed_df_2['problem_seq'].values:
                    selected_seq = seq
                    break
            if selected_seq is not None:
                rows_to_duplicate['problem_seq'] = selected_seq  
            else:
                rows_to_duplicate['problem_seq'] = rows_to_duplicate['problem_seq'].iloc[0]  
            rows_to_add.append(rows_to_duplicate)
if rows_to_add:
    detailed_df_2 = pd.concat([detailed_df_2] + rows_to_add, ignore_index=True)
detailed_df_2

,problem_seq,subject,big_category,middle_category,small_category,point,title_y,explain_text,open_dttm,response_count,response_summary,response_ratio
0,30118,지구과학Ⅰ,지구의 역사,퇴적암과 퇴적 구조,"퇴적암: 퇴적암의 생성, 퇴적암의 분류",3.0,420제 지구과학Ⅰ,None,2022-11-02,369.0,"0:76, 1:293","[0.21, 0.79]"
1,30119,지구과학Ⅰ,별과 외계 행성계,외계 생명체 탐사,생명 가능 지대,2.0,2023학년도 11월 고3 수능 과학탐구 (지구과학Ⅰ),2022년 11월 시행 | 20문제,2022-11-28,285.0,"0:66, 1:219","[0.23, 0.77]"
2,30120,지구과학Ⅰ,지권의 변동,마그마 생성,판 경계와 마그마 생성,2.0,420제 지구과학Ⅰ,None,2022-11-02,565.0,"0:148, 1:417","[0.26, 0.74]"
3,30121,지구과학Ⅰ,대기와 해양의 변화,태풍,태풍 통과 시의 기상 요소 변화,3.0,2023학년도 11월 고3 수능 과학탐구 (지구과학Ⅰ),None,2022-11-02,614.0,"0:195, 1:419","[0.32, 0.68]"
4,30122,지구과학Ⅰ,대기와 해양의 변화,기압과 날씨 변화,온대 저기압의 날씨,3.0,2023학년도 11월 고3 수능 과학탐구 (지구과학Ⅰ),None,2022-11-02,975.0,"0:304, 1:671","[0.31, 0.69]"
5,30125,지구과학Ⅰ,외부 은하와 우주 팽창,빅뱅 우주론,빅뱅 우주론,2.0,2023학년도 11월 고3 수능 과학탐구 (지구과학Ⅰ),2022년 11월 시행 | 20문제,2022-11-28,334.0,"0:142, 1:192","[0.43, 0.57]"
6,30131,지구과학Ⅰ,대기와 해양의 상호 작용,엘니뇨와 라니냐,"엘니뇨와 라니냐: 수온 편차 변화, 풍속 편차 변화, 기압 편차 변화, 적외선 복사...",2.0,2023학년도 11월 고3 수능 과학탐구 (지구과학Ⅰ),None,2022-11-02,328.0,"0:145, 1:183","[0.44, 0.56]"
7,33371,지구과학Ⅰ,대기와 해양의 상호 작용,기후 변화의 요인,기후 변화의 여러 가지 요인,2.0,2024학년도 6월 고3 모의평가 과학탐구 (지구과학Ⅰ),2023년 6월 시행 | 20문제,2023-06-07,238.0,"0:71, 1:167","[0.3, 0.7]"
8,33381,지구과학Ⅰ,별과 외계 행성계,별의 물리량,별의 물리량,3.0,2024학년도 6월 고3 모의평가 과학탐구 (지구과학Ⅰ),2023년 6월 시행 | 20문제,2023-06-07,556.0,"0:213, 1:343","[0.38, 0.62]"
9,33383,지구과학Ⅰ,별과 외계 행성계,외계 행성계 탐사,외계 행성계 탐사 방법: 식 현상과 시선 속도 변화 이용,3.0,2024학년도 6월 고3 모의평가 과학탐구 (지구과학Ⅰ),2023년 6월 시행 | 20문제,2023-06-07,542.0,"0:277, 1:265","[0.51, 0.49]"


In [12]:
filtered_data[filtered_data['problem_seq']==30729]

,member_idx,member_exam_seq,problem_seq,answer_dttm,correct_yn,title_y,explain_text,open_dttm,subject,big_category,...,small_category,ref_seq,point,answer_rate,set_type,type1,type2,num1,ref_num,represent_yn


In [13]:
asdfsad

NameError: name 'asdfsad' is not defined

In [ ]:

def get_frequent_value(series):
    value_counts = series.value_counts(dropna=True)
    if value_counts.empty:
        return None  
    if len(value_counts) > 1 and value_counts.iloc[0] == value_counts.iloc[1]:
        return series.dropna().iloc[0] if not series.dropna().empty else None
    return value_counts.idxmax()  

def map_first_occurrence(problem_seq):
    return problem_seq_mapping_dict.get(problem_seq, [None])[0]

# problem_seq_mapping_dict = {problem: idx + 1 for idx, problem in enumerate(sorted(all_top_standardized))}
problem_seq_mapping_dict = defaultdict(list)
for idx, problem in enumerate(all_top_standardized):
    problem_seq_mapping_dict[problem].append(idx + 1)

if set(subjects_list) & set(['독서','문학']):   
    print('\n 국어')
    mask = preprocessing_done['problem_seq'].isin(all_top_standardized)
    selected_data = preprocessing_done[mask].copy()
    selected_data['standardized_problem_seq_mapped'] = selected_data['problem_seq'].apply(map_first_occurrence)
else:
    mask = preprocessing_done['standardized_problem_seq'].isin(all_top_standardized)
    selected_data = preprocessing_done[mask].copy()
    selected_data['standardized_problem_seq_mapped'] = selected_data['standardized_problem_seq'].apply(map_first_occurrence)
# Aggregate data to find the most frequent 'correct_yn' values
aggregated_data = selected_data.groupby(['member_idx', 'standardized_problem_seq_mapped'])['correct_yn'].agg(get_frequent_value).reset_index()

# Generate full index DataFrame for all combinations of 'member_idx_mapped' and problem sequences from 1 to 10
standardized_problem_seq_list = np.arange(1, len(all_top_standardized)+1) # np.arange(1, 11) Create a range from 1 to 10 for problem sequences
full_index = pd.MultiIndex.from_product(
    [selected_data['member_idx'].unique(), standardized_problem_seq_list],
    names=['member_idx', 'standardized_problem_seq_mapped'])
full_data = pd.DataFrame(index=full_index).reset_index()
full_data = full_data.merge(aggregated_data, on=['member_idx', 'standardized_problem_seq_mapped'], how='left')
# duplicates 
for problem, indices in problem_seq_mapping_dict.items():
    if len(indices) > 1:  # More than one index mapped to the same problem
        # Get the source data from the first of the mapped indices
        source_index = indices[0]
        source_data = full_data[full_data['standardized_problem_seq_mapped'] == source_index][['member_idx', 'correct_yn']].set_index('member_idx')
        
        # Apply the source data to all other mapped indices
        for target_index in indices[1:]:
            condition = full_data['standardized_problem_seq_mapped'] == target_index
            full_data.loc[condition, 'correct_yn'] = full_data.loc[condition, 'member_idx'].map(source_data['correct_yn'])
full_data = full_data.sort_values(by=['member_idx', 'standardized_problem_seq_mapped'])

# add original column. 
full_data['original'] = np.where(full_data['correct_yn'].isna(), 'N', 'Y')

print("Standardized Problem Seq Mapping:")
print(problem_seq_mapping_dict) 
print(full_data.iloc[29:40,:])  # Display the first 11 rows to check the outcome

Standardized Problem Seq Mapping:
defaultdict(<class 'list'>, {25335: [1], 25194: [2], 25076: [3], 25074: [4], 25330: [5], 25342: [6], 36178: [7], 36168: [8], 25340: [9], 36161: [10]})
    member_idx  standardized_problem_seq_mapped  correct_yn original
29        4500                               10         NaN        N
30       20606                                1         1.0        Y
31       20606                                2         1.0        Y
32       20606                                3         1.0        Y
33       20606                                4         1.0        Y
34       20606                                5         1.0        Y
35       20606                                6         1.0        Y
36       20606                                7         1.0        Y
37       20606                                8         0.0        Y
38       20606                                9         0.0        Y
39       20606                               10         

In [ ]:

# ##### 응답률에 따른 학생 수 보기

def analyze_data_quality(full_data):
    # unique_problems = full_data['standardized_problem_seq_mapped'].nunique()
    unique_problems = len({index for sublist in problem_seq_mapping_dict.values() for index in sublist})
    print('unique_problems',unique_problems)
    results_df = pd.DataFrame(columns=["Threshold", "Remaining Member Count", "Filtered Out Member Count"])
    for threshold_percent in range(0, 100, 10):
        threshold = unique_problems * (threshold_percent / 100.0)
        nan_counts = full_data['correct_yn'].isna().groupby(full_data['member_idx']).sum()
        members_to_remove = nan_counts[nan_counts > threshold].index
        filtered_data = full_data[~full_data['member_idx'].isin(members_to_remove)]
        member_idx_mapping = {idx: i for i, idx in enumerate(filtered_data['member_idx'].unique())}
        filtered_data['member_idx'] = filtered_data['member_idx'].map(member_idx_mapping)
        remaining_member_count = filtered_data['member_idx'].nunique()
        filtered_out_member_count = full_data['member_idx'].nunique() - remaining_member_count
        new_row = pd.DataFrame([{
            "Threshold": f"{threshold_percent}%",
            "Remaining Member Count": remaining_member_count,
            "Filtered Out Member Count": filtered_out_member_count
        }])
        results_df = pd.concat([results_df, new_row], ignore_index=True)   
    return results_df

def data_filtering(full_data, threshold, num_problems):
    score_column = 'correct_yn'
    threshold_percent = threshold / 100.0  
    threshold = num_problems * threshold_percent 
    nan_counts = full_data[score_column].isna().groupby(full_data['member_idx']).sum()
    members_to_remove = nan_counts[nan_counts > threshold].index
    filtered_data = full_data[~full_data['member_idx'].isin(members_to_remove)]
    unique_members = pd.unique(filtered_data['member_idx'])
    unique_members.sort()  
    member_idx_mapping_dict = {member: idx + 1 for idx, member in enumerate(unique_members)}
    filtered_data['member_idx_mapped'] = filtered_data['member_idx'].map(member_idx_mapping_dict)
    filtered_data.sort_values(by=['member_idx_mapped', 'standardized_problem_seq_mapped'], inplace=True)
    return filtered_data, member_idx_mapping_dict

def process_member_NA1(member, full_data_merged, preprocessing_done_reduced):
    # print('41 member',member)
    updates = {}
    preprocessing_done_reduced = preprocessing_done_reduced[preprocessing_done_reduced['member_idx'] == member]
    member_data = full_data_merged[full_data_merged['member_idx'] == member]  # Use original member_idx
    # print('44 memberdata',member_data)
    for index, row in member_data.iterrows():
        # print('49 index',index, 'row',row)
        if pd.isna(row['correct_yn']):
            mask = (
                (preprocessing_done_reduced['subject'] == row['subject']) &
                (preprocessing_done_reduced['big_category'] == row['big_category']) &
                (preprocessing_done_reduced['middle_category'] == row['middle_category'])
            )
            filtered_attempts = preprocessing_done_reduced[mask]
            higher_points_correct = filtered_attempts[(filtered_attempts['point'] > row['point']) & (filtered_attempts['most_frequent'] == 1)]
            lower_points_incorrect = filtered_attempts[(filtered_attempts['point'] < row['point']) & (filtered_attempts['most_frequent'] == 0)]
            higher_count = len(higher_points_correct)
            lower_count = len(lower_points_incorrect)
            if higher_count > 0 or lower_count > 0:
                result = 1 if higher_count > lower_count else 0
                updates[index] = result
    return updates

def fill_nan_correct_yn(full_data, preprocessing_done, problem_seq_mapping_dict):
    inverse_mapping = {i: problem_seq for problem_seq, indices in problem_seq_mapping_dict.items() for i in indices}
    full_data['standardized_problem_seq'] = full_data['standardized_problem_seq_mapped'].map(inverse_mapping)
    #######################
    valid_data = preprocessing_done.dropna(subset=['correct_yn'])
    mode_df = valid_data.groupby(['member_idx', 'standardized_problem_seq'])['correct_yn'].agg(lambda x: x.value_counts().idxmax()).reset_index()
    mode_df.columns = ['member_idx', 'standardized_problem_seq', 'most_frequent']
    # print('modedf',mode_df, mode_df.shape)
    
    # print('\n 72',preprocessing_done[preprocessing_done['standardized_problem_seq']==9306])
    preprocessing_done_reduced = preprocessing_done.drop_duplicates(subset=['member_idx', 'problem_seq'])[['member_idx', 'problem_seq', 'correct_yn', 'subject',
       'big_category', 'middle_category', 'point','standardized_problem_seq']]
    # print('\n 75',preprocessing_done_reduced[preprocessing_done_reduced['standardized_problem_seq']==9306].shape)
    preprocessing_done_reduced = preprocessing_done_reduced.merge(mode_df, left_on=['member_idx', 'standardized_problem_seq'], right_on=['member_idx', 'standardized_problem_seq'], how='left')
    # print('\n 77',preprocessing_done_reduced[preprocessing_done_reduced['standardized_problem_seq']==9306].shape)
    # print('\n full_data',full_data, full_data.shape)
    # print('dtypes ',full_data.dtypes)
    full_data_merged = full_data.merge(
        preprocessing_done_reduced.drop_duplicates(subset=['standardized_problem_seq'], keep='first'), left_on=['standardized_problem_seq'], right_on=['standardized_problem_seq'], how='left', suffixes=('', '_drop'))

    full_data_merged.drop([col for col in full_data_merged.columns if 'drop' in col], axis=1, inplace=True)
    # print('full_data_merged',full_data_merged, full_data_merged.shape)
    members = full_data_merged['member_idx'].unique()
    with ProcessPoolExecutor(max_workers=32) as executor:
        futures = {executor.submit(process_member_NA1, member, full_data_merged, preprocessing_done_reduced): member for member in members}
        for future in futures:
            member_updates = future.result()
            for index, value in member_updates.items():
                if pd.isna(full_data_merged.at[index, 'correct_yn']):
                    full_data_merged.at[index, 'correct_yn'] = value

    return full_data_merged[['member_idx', 'standardized_problem_seq_mapped', 'correct_yn','original']]

In [ ]:
asdfsad

problem_seq_point_mapping_dict.json 만들기

Info Function을 위한 P_theta 값 찾기

In [ ]:

if set(subjects_list) & set(['독서','문학']): 
    standard = 'problem_seq'
else:
    standard = 'standardized_problem_seq'

preprocessing_done_answer_rate = preprocessing_done[
    preprocessing_done[standard].isin(problem_seq_mapping_dict.keys())
]

answer_rate_per_standard_problem_seq = preprocessing_done_answer_rate.groupby(standard)['answer_rate'].mean() / 100

p_theta_problem = defaultdict(list)
for problem_seq in problem_seq_mapping_dict:
    answer_rate = answer_rate_per_standard_problem_seq.get(problem_seq, np.nan)
    for index in problem_seq_mapping_dict[problem_seq]:
        p_theta_problem[index].append(answer_rate)

for key in list(p_theta_problem):
    p_theta_problem[key] = np.nanmean(p_theta_problem[key])  # Compute mean if multiple values exist, handling NaNs

dict(p_theta_problem)


{1: 0.7515151515151516,
 2: 0.5237777777777778,
 3: 0.8113043478260871,
 4: 0.7811538461538461,
 5: 0.6690909090909091,
 6: 0.485,
 7: 0.3655555555555556,
 8: 0.42588235294117643,
 9: 0.553076923076923,
 10: 0.7966666666666667}

In [ ]:
filtered_data_0, _= data_filtering(full_data,threshold=30, num_problems=len(all_top_standardized)) # NaN 비율이 30%가 넘어가면

prob_df = filtered_data_0[filtered_data_0['correct_yn'].isin([0, 1])]
group_counts = prob_df.groupby('standardized_problem_seq_mapped').size()

prob_df = prob_df.groupby('standardized_problem_seq_mapped')['correct_yn'].agg(
    count='count',
    count_1=lambda x: (x == 1).sum(),  # Count of 1s
    count_0=lambda x: (x == 0).sum(),  # Count of 0s
    probability=lambda x: (x == 1).sum() / x.count()
).reset_index()

p_theta_response = prob_df.set_index('standardized_problem_seq_mapped')['probability'].to_dict()

### p_theta_response_answer_ratio_counts
p_theta_response_answer_ratio_counts = prob_df.set_index('standardized_problem_seq_mapped')[['count_0', 'count_1']].apply(lambda x: [x['count_0'], x['count_1']], axis=1).to_dict()
print('p_theta_response',p_theta_response)
print('p_theta_response_answer_ratio_counts',p_theta_response_answer_ratio_counts)
### p_theta_response_answer_ratio_counts
p_theta_response_answer_ratio_counts = {int(k): [int(v[0]), int(v[1])] for k, v in p_theta_response_answer_ratio_counts.items()}
print('p_theta_response_answer_ratio_counts222222',p_theta_response_answer_ratio_counts)



p_theta_response {1: 0.5714285714285714, 2: 0.42857142857142855, 3: 0.625, 4: 0.625, 5: 0.7142857142857143, 6: 0.42857142857142855, 7: 0.6, 8: 0.0, 9: 0.42857142857142855, 10: 0.8333333333333334}
p_theta_response_answer_ratio_counts {1: [3, 4], 2: [4, 3], 3: [3, 5], 4: [3, 5], 5: [2, 5], 6: [4, 3], 7: [2, 3], 8: [7, 0], 9: [4, 3], 10: [1, 5]}
p_theta_response_answer_ratio_counts222222 {1: [3, 4], 2: [4, 3], 3: [3, 5], 4: [3, 5], 5: [2, 5], 6: [4, 3], 7: [2, 3], 8: [7, 0], 9: [4, 3], 10: [1, 5]}


In [ ]:
prob_df.set_index('standardized_problem_seq_mapped')['count'].to_dict()

{1: 7, 2: 7, 3: 8, 4: 8, 5: 7, 6: 7, 7: 5, 8: 7, 9: 7, 10: 6}

In [ ]:
asdf

NameError: name 'asdf' is not defined

In [ ]:

num_students_by_NaN_ratio = analyze_data_quality(full_data)
num_students_by_NaN_ratio

unique_problems 10


,Threshold,Remaining Member Count,Filtered Out Member Count
0,0%,137,2127
1,10%,145,2119
2,20%,165,2099
3,30%,215,2049
4,40%,291,1973
5,50%,388,1876
6,60%,511,1753
7,70%,763,1501
8,80%,1743,521
9,90%,2264,0


In [ ]:

filled_data1 = fill_nan_correct_yn(full_data, preprocessing_done, problem_seq_mapping_dict)
filled_data1

,member_idx,standardized_problem_seq_mapped,correct_yn,original
0,559,1,NaN,N
1,559,2,NaN,N
2,559,3,NaN,N
3,559,4,NaN,N
4,559,5,NaN,N
...,...,...,...,...
22635,135281,6,NaN,N
22636,135281,7,NaN,N
22637,135281,8,NaN,N
22638,135281,9,NaN,N


In [ ]:
analyze_data=analyze_data_quality(filled_data1)
analyze_data

unique_problems 10


,Threshold,Remaining Member Count,Filtered Out Member Count
0,0%,142,2122
1,10%,158,2106
2,20%,172,2092
3,30%,230,2034
4,40%,306,1958
5,50%,413,1851
6,60%,528,1736
7,70%,790,1474
8,80%,1861,403
9,90%,2264,0


In [ ]:
filtered_data_1,  member_idx_mapping_dict_1= data_filtering(filled_data1,threshold=30, num_problems=len(all_top_standardized)) # NaN 비율이 30%가 넘어가면

In [ ]:
def calculate_similarity(series1, series2):
    where_are_nans = np.logical_or(np.isnan(series1), np.isnan(series2))
    series1 = series1[~where_are_nans]
    series2 = series2[~where_are_nans]
    return (series1 == series2).sum()

def process_member_NA2(member, filled_data, non_nan_members):
    member_data = filled_data[filled_data['member_idx'] == member]
    nan_problems = member_data[member_data['correct_yn'].isna()]['standardized_problem_seq_mapped'].unique()
    results = []
    for problem in nan_problems:
        member_responses = member_data[member_data['standardized_problem_seq_mapped'] != problem]['correct_yn']
        similarity_scores = {}
        for other_member in non_nan_members:
            other_member_data = filled_data[filled_data['member_idx'] == other_member]
            other_member_responses = other_member_data[other_member_data['standardized_problem_seq_mapped'] != problem]['correct_yn']

            if len(member_responses) == len(other_member_responses):
                similarity = calculate_similarity(np.array(member_responses), np.array(other_member_responses))
                similarity_scores[other_member] = similarity

        if similarity_scores:
            # print('23 similarity scores',similarity_scores)
            most_similar_members = [member for member, score in similarity_scores.items() if score == max(similarity_scores.values())]
            most_similar_responses = filled_data[(filled_data['member_idx'].isin(most_similar_members)) & 
                                                  (filled_data['standardized_problem_seq_mapped'] == problem)]['correct_yn']
            # print('27 ', most_similar_members,'\n res', most_similar_responses, most_similar_responses.dtype)
            # print('########################################')
            # print('filled_data',filled_data)
            try:
                mode_response = mode(most_similar_responses, nan_policy='omit')
                # print('mode response',mode_response,'mode_response.mode',mode_response.mode)
                mode_value = mode_response.mode 
            except IndexError:
                # print('index error')
                mode_value = mode_response.mode[0] if mode_response.count[0] > 0 else np.nan
            results.append((member, problem, mode_value))

    return results
def identify_non_nan_members(filled_data):
    nan_counts = filled_data.groupby('member_idx')['correct_yn'].apply(lambda x: x.isna().sum())
    non_nan_members = nan_counts[nan_counts <= len(all_top_standardized)*0.4].index
    return non_nan_members

def fill_remaining_nans(filled_data):
    filled_data.sort_values(by=['member_idx', 'standardized_problem_seq_mapped'], inplace=True)
    
    # Identifying members with NaN values in 'correct_yn'
    nan_members = filled_data[filled_data['correct_yn'].isna()]['member_idx'].unique()
    
    # Identifying members without NaNs who are not in nan_members
    candidate_members = identify_non_nan_members(filled_data)
    non_nan_members = np.setdiff1d(candidate_members, nan_members)
    
    with ProcessPoolExecutor(max_workers=32) as executor:
        futures = {}
        for member in tqdm(nan_members, desc="Processing members"):
            future = executor.submit(process_member_NA2, member, filled_data, non_nan_members)
            futures[future] = member
        
        for future in tqdm(futures, desc="Completing tasks"):
            results = future.result()
            for member, problem, value in results:
                # print('58 member',member, 'problem',problem,'value', value)
                # Check if 'correct_yn' is NaN before updating
                condition = ((filled_data['member_idx'] == member) & 
                             (filled_data['standardized_problem_seq_mapped'] == problem))
                # Apply the mode only where 'correct_yn' is NaN
                filled_data.loc[condition & filled_data['correct_yn'].isna(), 'correct_yn'] = value

    return filled_data

# Assuming filled_data is the DataFrame after the first filling step
filled_data2 = fill_remaining_nans(filled_data1)

# Optionally, save the final filled data to a new CSV
# filled_data2.to_csv('./filled_data_2.csv', encoding='utf-8-sig', index=False)

Completing tasks: 100%|██████████| 2122/2122 [00:50<00:00, 42.33it/s]


In [ ]:
analyze_data=analyze_data_quality(filled_data2)
analyze_data


unique_problems 10


,Threshold,Remaining Member Count,Filtered Out Member Count
0,0%,2264,0
1,10%,2264,0
2,20%,2264,0
3,30%,2264,0
4,40%,2264,0
5,50%,2264,0
6,60%,2264,0
7,70%,2264,0
8,80%,2264,0
9,90%,2264,0


In [ ]:
filtered_data_2, member_idx_mapping_dict_2 = data_filtering(filled_data2,threshold=30, num_problems=len(all_top_standardized)) # NaN 비율이 30%가 넘어가면

In [ ]:
filtered_data_2

,member_idx,standardized_problem_seq_mapped,correct_yn,original,member_idx_mapped
0,559,1,1.0,N,1
1,559,2,0.0,N,1
2,559,3,0.0,N,1
3,559,4,0.0,N,1
4,559,5,0.0,N,1
...,...,...,...,...,...
22635,135281,6,0.0,N,2264
22636,135281,7,1.0,N,2264
22637,135281,8,0.0,N,2264
22638,135281,9,1.0,N,2264
